In [1]:
import pandas as pd
from dotenv import load_dotenv

In [15]:
train_data = pd.read_csv(r'/workspaces/Mlops_Project/gcp_data/train.csv')

In [3]:
train_data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
train_data['cut'].value_counts()

cut
Ideal        92454
Premium      49910
Very Good    37566
Good         11622
Fair          2021
Name: count, dtype: int64

In [5]:
#dropping id column
train_data.drop(labels=["id"],axis=1,inplace=True)

In [6]:
cat_columns = train_data.columns[train_data.dtypes=='object']

In [7]:
num_columns = train_data.columns[train_data.dtypes !='object']

In [8]:
# oridinal encoding on categorical
cut_map={"Fair":1,"Good":2,"Very Good":3,"Premium":4,"Ideal":5}
clarity_map = {"I1":1,"SI2":2 ,"SI1":3 ,"VS2":4 , "VS1":5 , "VVS2":6 , "VVS1":7 ,"IF":8}
color_map = {"D":1 ,"E":2 ,"F":3 , "G":4 ,"H":5 , "I":6, "J":7}

In [9]:
train_data["cut"]=train_data["cut"].map(cut_map)
train_data["clarity"]=train_data["clarity"].map(clarity_map)
train_data["color"]=train_data["color"].map(color_map)

In [10]:
train_data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,4,3,4,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,3,7,2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,5,4,5,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,5,4,5,61.6,56.0,4.38,4.41,2.71,666
4,1.70,4,4,4,62.6,59.0,7.65,7.61,4.77,14453


In [11]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler # HAndling Feature Scaling
from sklearn.preprocessing import OrdinalEncoder # Ordinal Encoding
## pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [18]:
x = train_data.drop(labels=['price','id'],axis=1)
y = train_data["price"]

In [19]:
x

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77
...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81


In [20]:
x_cat_cols = x.select_dtypes(include="object")
x_num_cols = x.select_dtypes(exclude="object")

In [23]:
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [24]:
num_pipeline = Pipeline(
    steps=[("imputer",SimpleImputer()),
           ("scaler",StandardScaler())]
)

In [25]:
cat_pipeline = Pipeline(steps=[("imputer",SimpleImputer(strategy='most_frequent')),
                               ("ordinalencoder",OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))])

In [34]:
cat_cols=x.select_dtypes(include="object").columns

num_cols=x.select_dtypes(exclude="object").columns

In [35]:
preprocessor = ColumnTransformer([("num_pipeline",num_pipeline,num_cols),("cat_pipeline",cat_pipeline,cat_cols)])

In [28]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test  = train_test_split(x,y,test_size=0.3,random_state=42)

In [36]:
preprocessor.fit_transform(x_train)

array([[-0.82314374, -1.12998781, -0.64189666, ...,  4.        ,
         1.        ,  5.        ],
       [ 0.94502267, -1.77782269,  0.92190185, ...,  2.        ,
         4.        ,  4.        ],
       [ 1.9584839 ,  0.16568195,  0.40063568, ...,  3.        ,
         4.        ,  3.        ],
       ...,
       [ 0.92345966,  0.90606467,  0.40063568, ...,  3.        ,
         3.        ,  3.        ],
       [-1.03877378, -0.66724861, -0.64189666, ...,  2.        ,
         3.        ,  6.        ],
       [-1.03877378, -0.01941373,  0.92190185, ...,  2.        ,
         3.        ,  1.        ]])

In [37]:
x_train=pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test=pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [39]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [40]:
models = {
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet(),
    'Randomforest':RandomForestRegressor(),
    'xgboost':XGBRegressor()
}

In [47]:
trained_model_list = []
model_list = []
r2_list = []

In [42]:
list(models)

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet', 'Randomforest', 'xgboost']

In [43]:
def evaluate_model(true,pred):
    r2 = r2_score(true,pred),
    mae = mean_absolute_error(true,pred),
    mse = mean_squared_error(true,pred)
    return mae,mse,r2

In [48]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    #fitting the model
    model.fit(x_train,y_train)

    #make prediction
    y_pred = model.predict(x_test)

    MAE,MSE,R2=evaluate_model(y_test,y_pred)

    print("model training performance for:",model)
    print("MSE:",MSE)
    print("MAE:",MAE)
    print("R2:",R2)
    
    r2_list.append(R2)

    print("="*40)
    print("\n")


model training performance for: LinearRegression()
MSE: 1029473.3531156892
MAE: (675.075827006745,)
R2: (0.9362906819996045,)


model training performance for: Lasso()
MSE: 1029534.1175907082
MAE: (676.378880308673,)
R2: (0.936286921568846,)


model training performance for: Ridge()
MSE: 1029482.8329815964
MAE: (675.1087261318496,)
R2: (0.936290095334792,)


model training performance for: ElasticNet()
MSE: 2291130.0570947696
MAE: (1051.172228186302,)
R2: (0.8582126162411591,)


model training performance for: RandomForestRegressor()
MSE: 372982.25302129745
MAE: (310.0247162595974,)
R2: (0.9769178630079923,)


model training performance for: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
           